In [1]:
! pip install pydicom
!pip install dicom2nifti

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 68.6 MB/s eta 0:00:00


In [ ]:
import pydicom
import os
import dicom2nifti
import nibabel as nib
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
pacientes = [] #Se almacenan los pacientes
imagenes_originales = {}  # Almacena imágenes originales
imagenes_procesadas = {}  # Almacena imágenes procesadas

class DICOM:

    def __init__(self, edad, id, nifti):
        self.edad = edad
        self.id = id
        self.nifti = nifti

    @classmethod
    def Pacientes(self, pacientes):
        carpeta = "Datos"
        output_nifti = "Niftii"
        if not os.path.exists(output_nifti):
            os.makedirs(output_nifti)
        dicom_slices = []
        positions = []
        for archivo in os.listdir(carpeta):
            archivo_path = os.path.join(carpeta, archivo)
            if archivo.endswith(".dcm"):
                ds = pydicom.dcmread(archivo_path)
                id = ds.PatientID
                edad = ds.PatientAge
                pixel_data = ds.pixel_array
                dicom_slices.append(pixel_data)
                positions.append(float(ds.ImagePositionPatient[2]))
                sorted_indices = np.argsort(positions)
                dicom_slices = [dicom_slices[i] for i in sorted_indices]
                volume = np.stack(dicom_slices, axis=-1)
                nifti_image = nib.Nifti1Image(volume, affine=np.eye(4))
                self.a = f"{id}.nii.gz"
                output_path = os.path.join(output_nifti, self.a)
                nib.save(nifti_image, output_path)
                print(f"Archivo NIfTI guardado en: {output_path}")
                paciente = self(edad, id, output_path)
                pacientes.append(paciente)

    @classmethod
    def mostrar_pacientes(self, pacientes):
        for paciente in pacientes:
            print(f"ID: {paciente.id}, EDAD: {paciente.edad}, NIFTI: {self.a}")

    @classmethod
    def Rotar_Imagen(self):
        while True:
          dicom = input("Ingrese nombre o ruta del DICOM: ")
          dicom_data = pydicom.dcmread(dicom)
          imagen = dicom_data.pixel_array
          angulo = int(input("Ingrese el ángulo de rotación (90, 180, 270): "))
          if angulo not in [90, 180, 270]:
              print("El ángulo debe ser uno de los recomendados. Intente nuevamente")
              continue
          else:
              row, col= np.shape(imagen)
              MR = cv2.getRotationMatrix2D((col/2,row/2),angulo,1)
              rot = cv2.warpAffine(imagen,MR,(col,row))
              plt.figure(figsize=(10, 5))

              #Original
              plt.subplot(1, 2, 1)
              plt.imshow(imagen, cmap='gray')
              plt.title('Imagen Original')
              plt.axis('off')

              # Imagen rotada
              plt.subplot(1, 2, 2)
              plt.imshow(rot, cmap='gray')
              plt.title(f'Imagen Rotada {angulo} grados')
              plt.axis('off')

              plt.show()

              # Guardar la imagen
              nombre_salida = "imagen_rotada.png"
              cv2.imwrite(nombre_salida, rot)
              print(f'Imagen guardada como: {nombre_salida}')
              break



    def cargar_imagen():
        """Permite al usuario cargar una imagen JPG o PNG."""
        ruta = input("Ingrese la ruta de la imagen (JPG o PNG): ").strip()
        nombre = ruta.split("/")[-1] # no se que ruta vamos a usar al final
        img = cv2.imread(ruta)
        if img is None:
            print("Error: No se pudo cargar la imagen. Verifique la ruta.")
            return
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        imagenes_originales[nombre] = img_gray
        print(f"Imagen '{nombre}' cargada exitosamente.")

    def procesar_imagen(imagenes_originales, imagenes_procesadas):
        """Permite procesar una imagen cargada, aplicar binarizacion y transformación morfologica."""
        if not imagenes_originales:
            print("Error: No hay imagenes cargadas. Cargue una imagen primero.")
            return

        # Seleccionar imagen
        print("Imágenes disponibles:")
        for i, nombre in enumerate(imagenes_originales.keys()):
            print(f"{i + 1}. {nombre}") #odio trabajar con diccionarios
        opcion = int(input("Seleccione el número de la imagen a procesar: "))
        nombre = list(imagenes_originales.keys())[opcion - 1]
        img_gray = imagenes_originales[nombre]

        # Binarización    #gracias astrid por el documento
        umbral = int(input("Ingrese el umbral para binarización (0-255): "))
        _, img_bin = cv2.threshold(img_gray, umbral, 255, cv2.THRESH_BINARY)

        # Transformacion morfologica    # x2 gracias astrid
        print("Ingrese el tamaño del kernel (e.g., 3 para un kernel 3x3): ")
        tam_kernel = int(input("Tamaño del kernel: "))
        kernel = np.ones((tam_kernel, tam_kernel), np.uint8)
        img_morph = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, kernel)

        #para Agregar texto usando openCV GOD como lo pide el punto
        img_color = cv2.cvtColor(img_morph, cv2.COLOR_GRAY2BGR)
        texto = f"Imagen binarizada | Umbral: {umbral} | Kernel: {tam_kernel}x{tam_kernel}"
        cv2.putText(img_color, texto, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        # Guardar imagen procesada
        nombre_procesado = f"procesado_{nombre}"
        cv2.imwrite(nombre_procesado, img_color)
        imagenes_procesadas[nombre_procesado] = img_color
        print(f"Imagen procesada guardada como '{nombre_procesado}'.")

    def mostrar_menu():
        """Despliega el menú principal del programa."""
        while True:
            opcion = input("Seleccione una opción: ")
            if opcion == "1":
                DICOM.Pacientes(pacientes)
            elif opcion == "2":
                DICOM.Rotar_Imagen()
            elif opcion == "3":
                DICOM.cargar_imagen()
            elif opcion == "4":
                DICOM.procesar_imagen(imagenes_originales, imagenes_procesadas)
            elif opcion == "5":
                print("Saliendo del programa...")
                break
            else:
                print("Opción no válida. Intente de nuevo.")
print("\nMenú:")
print("1. Ingresar paciente")
print("2. Rotar imagen")
print("3. Cargar imagen JPG o PNG")
print("4. Procesar imagen")
print("5. Salir")

# Ejecutar el programa
if __name__ == "__main__":
  DICOM.mostrar_menu()



Menú:
1. Ingresar paciente
2. Rotar imagen
3. Cargar imagen JPG o PNG
4. Procesar imagen
5. Salir
